In [1]:
from openai import OpenAI
import json
import random
import re
from goat_planner.utils.bt_validator import validate_behavior_tree
from prompt_data import action_list, object_list
random.seed(42)

In [3]:
def generate_query_prompt(object_context, action_context, last_queries):
    base_prompt = """
You are simulating a human planning a task for a robot. The robot is equipped with sensors, manipulators, and mobility capabilities, and can perform tasks based on the objects in its environment and the actions available. It can also plan multi-step tasks or respond to impossible queries by indicating they cannot be completed.

Object Context: 
{object_context}

Actions Context:
{action_context}

Generate a human-like query for the robot based on the given context and actions. Ensure that the queries vary in complexity:
- Some queries should be straightforward (e.g., single-step tasks).
- Some queries should be multi-step or involve reasoning (e.g., combining multiple actions or making decisions based on object attributes).
- Occasionally, generate queries that are impossible to execute (e.g., due to missing objects, incompatible actions, or physical limitations).

Here is the list of old queries that you created:
{last_queries}

Be creative and simulate realistic scenarios. The queries should be natural and diverse, for example:
- Go find something to eat that's not too salty.
- Close the hot water valve in the kitchen.
- Go make dinner and set the table.
- Help the firefighter put out the fire by spraying water on it.

Generate only a single query.
"""

    full_prompt = base_prompt.format(
        object_context=object_context,
        action_context=action_context,
        last_queries="\n".join(last_queries)
    )
    return full_prompt, object_context


last_queries = ["no last query"]

query, reduced_object_list = generate_query_prompt(object_list, action_list, last_queries)

In [4]:
def generate_bt_prompt(object_context, action_context, query):
    base_prompt = """
You are a robot that generate Behavior Tree with actions and context in a ROS2-compatible environment. The robot is equipped with sensors, manipulators, and mobility capabilities, and can execute tasks using behavior trees.

Object Context: 
{object_context}

Actions Context:
{action_context}

Generate a Behavior Tree XML structure based on the given context and actions. The tree should:
- Include nodes for sequence (`Sequence`), fallback (`Fallback`), and actions based on the actions dictionary.
- Use realistic action nodes with specified parameters for the given scenario.
- Allow for graceful recovery from failures using fallback nodes.
- If the query is not possible to execute, you must explain why and return an empty Behavior Tree XML.


Detail what you are going to do and then generate a Behavior Tree XML for it. Exemple for the query: "Go find something to eat that's not too salty."

You must respect the following format:

[Explanation]
Good, I will check in my database which objects match the query. If there is a good match, I will navigate to it and pick it up. Otherwise, I will try to find a kitchen, open the fridge, and try to find something to eat that's not too salty. Then I'll bring it back to the table.

[Behavior Tree]
<root main_tree_to_execute="FindAndRetrieveFoodSequence">
    <BehaviorTree ID="FindAndRetrieveFoodSequence">
        <Sequence name="FindAndRetrieveFood">
            <Retry num_attempts="3">
                <Locate object="food not too salty" 
                        position_x="{{food_position_x}}" 
                        position_y="{{food_position_y}}" 
                        position_z="{{food_position_z}}" />
            </Retry>
            <Fallback name="HandleFoodSearchFallback">
                <Sequence name="DirectFoodRetrieval">
                    <Retry num_attempts="2">
                        <Navigate x="{{food_position_x}}" y="{{food_position_y}}" />
                    </Retry>
                    <Retry num_attempts="2">
                        <Pick object="food not too salty" 
                              grip_strength="medium" 
                              precision="high" />
                    </Retry>
                </Sequence>
                <Sequence name="KitchenSearchAndRetrieve">
                    <Retry num_attempts="2">
                        <Navigate x="{{kitchen_position_x}}" y="{{kitchen_position_y}}" />
                    </Retry>
                    <Retry num_attempts="3">
                        <Locate object="fridge" 
                                position_x="{{fridge_position_x}}" 
                                position_y="{{fridge_position_y}}" 
                                position_z="{{fridge_position_z}}" />
                    </Retry>
                    <Retry num_attempts="2">
                        <Navigate x="{{fridge_position_x}}" y="{{fridge_position_y}}" />
                    </Retry>
                    <Retry num_attempts="3">
                        <Locate object="food not too salty" 
                                position_x="{{food_position_x}}" 
                                position_y="{{food_position_y}}" 
                                position_z="{{food_position_z}}" />
                    </Retry>
                    <Retry num_attempts="2">
                        <Pick object="food not too salty" 
                              grip_strength="medium" 
                              precision="high" />
                    </Retry>
                </Sequence>
            </Fallback>
            <Retry num_attempts="2">
                <Navigate x="{{table_position_x}}" y="{{table_position_y}}" />
            </Retry>
            <Retry num_attempts="2">
                <Place object="food not too salty" 
                       surface="table" 
                       orientation="upright" 
                       alignment="center" />
            </Retry>
        </Sequence>
    </BehaviorTree>
</root>


The answer must contains only those two parts (Explanation and Behavior Tree XML). 
Query: {query}
""" # Generate samples for object context and actions 
  
    full_prompt = base_prompt.format(
      object_context=object_context,
      action_context=action_context,
      query=query
      )
    return full_prompt


query = "Go find something to eat that's not too salty."
bt_prompt = generate_bt_prompt(reduced_object_list, action_list, query)

In [5]:
def generate_query(query_prompt):
    client = OpenAI()

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": query_prompt}],
        max_tokens=1000,
        temperature=0.7
    )
    return response.choices[0].message.content.strip()


In [6]:
def extract_explanation_and_bt(bt_prompt):
    explanation_match = re.search(r"\[Explanation\]\n(.*?)\n\[Behavior Tree\]", bt_prompt, re.S)
    bt_match = re.search(r"\[Behavior Tree\]\n(.*)", bt_prompt, re.S)  # Adjusted to capture everything after [Behavior Tree]
    
    explanation = explanation_match.group(1).strip() if explanation_match else ""
    bt = bt_match.group(1).strip() if bt_match else ""
    return explanation, bt

explanation, bt = extract_explanation_and_bt(bt_prompt)
# print(explanation)
# print(bt)

In [7]:
from goat_planner.utils.json_xml_convertor import xml_to_json


def generate_dataset(object_context, action_list, dataset_size=20):
    dataset = []
    last_queries = []  # To store the last 10 queries
    
    for _ in range(dataset_size):
        # Randomly select captions and actions
        reduced_object_list = random.sample(object_context, random.randint(3, 12))
        
        # Generate query
        query_prompt, _ = generate_query_prompt(reduced_object_list, action_list, last_queries)
        query = generate_query(query_prompt)
        bt_prompt = generate_bt_prompt(reduced_object_list, action_list, query)
        answer = generate_query(bt_prompt)
        explanation, bt_xml = extract_explanation_and_bt(answer)


        action_node = ["Sequence", "Fallback", "Retry"] + [action["name"] for action in action_list] if action_list else []

        is_valid = validate_behavior_tree(bt_xml, "xml", allowed_nodes=action_node)

        save = True
        if not is_valid[0]:
            print(f"Invalid Behavior Tree generated for query: {bt_xml}")
            print(f"Error: {is_valid[1]}")
            save = False
        else:
            bt_json = xml_to_json(bt_xml)

            is_valid = validate_behavior_tree(json.dumps(bt_json), "json", allowed_nodes=action_node)

            if not is_valid[0]:
                print(f"Invalid JSON conversion for query: {bt_json}")
                print(f"Invalid JSON conversion : {is_valid[1]}")
                save = False
            
            if save:
                print(f"Query {query} generated successfully")

                # Update the last queries
                last_queries.append(query)
                if len(last_queries) > 10:
                    last_queries.pop(0)  # Keep only the last 10 queries
                
                # Save the data entry
                dataset_entry = {
                    "object_context": reduced_object_list,
                    "actions_dictionary": action_list,
                    "query": query,
                    "explanation": explanation,
                    "bt_xml": bt_xml,
                    "bt_json": bt_json
                }
                dataset.append(dataset_entry)
            

    
    print(f"Dataset of {dataset_size} entries generated")
    return dataset

# Generate the dataset
dataset = generate_dataset(object_list, action_list, dataset_size=200)

Query Please locate the first aid kit and bring it to me. generated successfully
Query Could you please locate the set of screwdrivers and bring it to me, then place it on the countertop? generated successfully
Query Could you please navigate to the ladder, pick it up with medium grip strength, and then place it upright next to the stack of wood? generated successfully
Query Could you please navigate to the wooden cutting board on the dining table, pick up the silver knife with medium grip strength, and then place the knife upright on the counter? generated successfully
Query Please locate the crowbar, pick it up with high grip strength, and then navigate to the car jack to place the crowbar next to it. generated successfully
Query Could you please navigate to the cooking pot, pick it up with medium grip strength, and then place it on the stove in an upright position? generated successfully
Query Could you please locate the thermal blanket, pick it up with low grip strength, and then n

In [8]:
dataset[4]["explanation"]

"Good, I will start by locating the crowbar using my sensors. Once I have its position, I will navigate to it and pick it up with high grip strength. After that, I will navigate to the car jack's location and place the crowbar next to it."

In [9]:
save_path="./queries_dataset2.json"
# Save the dataset to a JSON file
with open(save_path, "w") as file:
    json.dump(dataset, file, indent=4)